In [ ]:
def get_asterisks_for_pval(p_val):
    """Receives the p-value and returns asterisks string."""
    if p_val > 0.05:
        p_text = "ns"  # above threshold => not significant
    elif p_val < 1e-4:  
        p_text = '****'
    elif p_val < 1e-3:
        p_text = '***'
    elif p_val < 1e-2:
        p_text = '**'
    else:
        p_text = '*'
    
    return p_text
        
def chisq_and_posthoc_corrected_total(df):
    """Receives a dataframe and performs chi2 test and then post hoc.
    Prints the p-values and corrected p-values (after FDR correction)"""
    # print(df)
    chi2, p, dof, ex = chi2_contingency(df, correction=True)
    # print(f"Chi2 result of the contingency table: {chi2}, p-value: {p}")
    return p

def t_test_overall_dataset(first, second, column):

    #First is the datafram of interest that we care about checking the distribution of
    #Second is the randomly selected datafram of our entire sample
    #Column is whatever catagorical column we are checking the distribution of

    p_vals = []

    #Get the non NA values from the desired column of the dataset
    notes = first.loc[:,column]
    notes = notes[notes.notna()]

    #Get the non NA values from the desired column of the random sample
    rand = second.loc[:,column]
    rand = rand[rand.notna()]

    #Print mean and st and run a ttest
    print(np.mean(notes), np.std(notes))
    print(np.mean(rand), np.std(rand))
    s, p = stats.ttest_ind(notes, rand)
    
    # print(f"T-test result: p-value {p}")
    return p

def catagorical_test_overall_dataset(first, second, column, column_index = None):

    #First is the datafram of interest that we care about checking the distribution of
    #Second is the randomly selected datafram of our entire sample
    #Column is whatever catagorical column we are checking the distribution of

    #I cant remember exactly what this does.....
    #I think the initial index variable are the catagories within the column
    if column_index == None:
        temp = pd.value_counts(first.loc[:,column])
        temp = temp.to_frame()
        initial_index = list(temp.index)
    else:
        initial_index = column_index

    # Create the main dataframe to run the chisquared test on. 
    # Notes in this case is just the column heading for the main dataset,
    # and rand is the column heading for the random dataset
    catagories = pd.DataFrame(0, index = initial_index, columns = ["Notes", "Rand"])
    
    #Gets the number of each catagory for the main dataset
    patients = pd.value_counts(first.loc[:,column])
    patients = patients.to_frame()
    patients.columns = ["Notes"]
    catagories.update(patients)

    #Gets the number of each catagory for the random dataset
    patients = pd.value_counts(second.loc[:,column])
    patients = patients.to_frame()
    patients.columns = ["Rand"]
    catagories.update(patients)
    

    return chisq_and_posthoc_corrected_total(catagories)
    